In [2]:
import pandas as pd

# Read in each csv file
double check variable types and ranges look good

In [91]:
#file_list = ["run_number.csv","daq_status.csv","cbrem_plane.csv","goni_roll.csv","goni_roll_rbv.csv","goni_x.csv","goni_x_rbv.csv","goni_y.csv","goni_y_rbv.csv","goni_pitch.csv","goni_pitch_rbv.csv","goni_yaw.csv","goni_yaw_rbv.csv"]
#col_names = ['RunNumber','DAQ:STATUS','CBREM:PLANE','GONI:ROLL','GONI:ROLL.RBV','GONI:X','GONI:X.RBV','GONI:Y','GONI:Y.RBV','GONI:PITCH','GONI:PITCH.RBV','GONI:YAW','GONI:YAW.RBV']

# ditching readfback values so we can define the nudges (as the moment the set point was changed)
file_list = ["run_number.csv","daq_status.csv","cbrem_plane.csv","goni_roll.csv","goni_x.csv","goni_y.csv","goni_pitch.csv","goni_yaw.csv"]
col_names = ['RunNumber','DAQ:STATUS','CBREM:PLANE','GONI:ROLL','GONI:X','GONI:Y','GONI:PITCH','GONI:YAW']
print(len(file_list))
print(len(col_names))

8
8


In [92]:
df_list = []

# this will create 13 dateframes for the 13 files listed above 
for i in range(len(file_list)):
    df_tmp = pd.read_csv("csv_data/initial_converted/"+file_list[i])

    df_tmp.columns = ['Date-Time',col_names[i]]
    print(df_tmp.head())
    print(df_tmp.describe())
    print(df_tmp.info())

    df_list.append(df_tmp)

             Date-Time  RunNumber
0  2020-01-11T01:26:54      71351
1  2020-01-11T03:25:42      71352
2  2020-01-11T04:16:50      71353
3  2020-01-11T06:18:46      71354
4  2020-01-11T08:38:06      71355
          RunNumber
count   1912.000000
mean   72310.573745
std      553.399577
min    71351.000000
25%    71833.750000
50%    72311.500000
75%    72789.250000
max    73267.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1912 entries, 0 to 1911
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Date-Time  1912 non-null   object
 1   RunNumber  1912 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 30.0+ KB
None
             Date-Time  DAQ:STATUS
0  2020-01-11T01:21:06           0
1  2020-01-11T01:26:55           1
2  2020-01-11T01:33:45           2
3  2020-01-11T03:04:37           0
4  2020-01-11T03:25:40           1
        DAQ:STATUS
count  5311.000000
mean      1.016193
std       0.812636
min       0.

# Merge small dataframes into big dataframe
Do outer join on Date-Time, most variables can be forward-filled. 

In [93]:
print(df_list[0].head())

             Date-Time  RunNumber
0  2020-01-11T01:26:54      71351
1  2020-01-11T03:25:42      71352
2  2020-01-11T04:16:50      71353
3  2020-01-11T06:18:46      71354
4  2020-01-11T08:38:06      71355


In [94]:
print(df_list[1].head())

             Date-Time  DAQ:STATUS
0  2020-01-11T01:21:06           0
1  2020-01-11T01:26:55           1
2  2020-01-11T01:33:45           2
3  2020-01-11T03:04:37           0
4  2020-01-11T03:25:40           1


In [95]:
import functools as ft 

In [96]:
df_final = ft.reduce(lambda left, right: pd.merge(left,right, on='Date-Time',how='outer'),df_list)

In [97]:
print(df_final.head())

             Date-Time  RunNumber  DAQ:STATUS  CBREM:PLANE  GONI:ROLL  GONI:X  \
0  2020-01-11T01:21:06        NaN         0.0          NaN        NaN     NaN   
1  2020-01-11T01:21:55        NaN         NaN          NaN        NaN     NaN   
2  2020-01-11T01:21:57        NaN         NaN          0.0        NaN     NaN   
3  2020-01-11T01:22:23        NaN         NaN          2.0        NaN     NaN   
4  2020-01-11T01:24:22        NaN         NaN          NaN        NaN     NaN   

   GONI:Y  GONI:PITCH  GONI:YAW  
0     NaN         NaN       NaN  
1     NaN    0.354557  0.773983  
2     NaN         NaN       NaN  
3     NaN         NaN       NaN  
4     NaN    1.830760  0.855300  


In [98]:
# these are all set points for the controls 
# nans occur during outer join when there is no updated value for a column 
# ie there is no new setpoint, so forward filling is appropriate here
df_test = df_final.ffill() 

In [99]:
# any missing variables handled by backfill (should just be the first few lines of the df)
df_test = df_test.bfill() # may make sense to drop first rows of nans too

In [102]:
df_test.head()

,Date-Time,RunNumber,DAQ:STATUS,CBREM:PLANE,GONI:ROLL,GONI:X,GONI:Y,GONI:PITCH,GONI:YAW
0,2020-01-11T01:21:06,71351.0,0.0,0.0,0.0,164.5,13.19,0.354557,0.773983
1,2020-01-11T01:21:55,71351.0,0.0,0.0,0.0,164.5,13.19,0.354557,0.773983
2,2020-01-11T01:21:57,71351.0,0.0,0.0,0.0,164.5,13.19,0.354557,0.773983
3,2020-01-11T01:22:23,71351.0,0.0,2.0,0.0,164.5,13.19,0.354557,0.773983
4,2020-01-11T01:24:22,71351.0,0.0,2.0,0.0,164.5,13.19,1.830760,0.855300


In [103]:
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17342 entries, 0 to 17341
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date-Time    17342 non-null  object 
 1   RunNumber    17342 non-null  float64
 2   DAQ:STATUS   17342 non-null  float64
 3   CBREM:PLANE  17342 non-null  float64
 4   GONI:ROLL    17342 non-null  float64
 5   GONI:X       17342 non-null  float64
 6   GONI:Y       17342 non-null  float64
 7   GONI:PITCH   17342 non-null  float64
 8   GONI:YAW     17342 non-null  float64
dtypes: float64(8), object(1)
memory usage: 1.2+ MB
None


In [104]:
df_test.describe()

,RunNumber,DAQ:STATUS,CBREM:PLANE,GONI:ROLL,GONI:X,GONI:Y,GONI:PITCH,GONI:YAW
count,17342.000000,17342.000000,17342.000000,17342.000000,17342.000000,17342.000000,17342.000000,17342.000000
mean,72136.116769,0.615904,1.011879,6.878564,143.364508,58.392777,0.493379,1.522049
std,572.333301,0.774147,0.778271,20.297506,39.979366,23.817149,1.287622,1.352042
min,71351.000000,0.000000,0.000000,-21.500000,0.000000,0.000000,-5.000000,-5.000000
25%,71622.000000,0.000000,0.000000,-10.500000,142.500000,58.100000,-0.734710,0.773983
50%,71959.000000,0.000000,1.000000,0.000000,156.000000,72.000000,0.377610,1.000000
75%,72683.000000,1.000000,2.000000,34.500000,161.500000,73.500000,0.764000,2.441610
max,73267.000000,2.000000,2.000000,50.000000,165.000000,76.300000,9.000000,9.000000


In [105]:
df_test.dtypes

Date-Time       object
RunNumber      float64
DAQ:STATUS     float64
CBREM:PLANE    float64
GONI:ROLL      float64
GONI:X         float64
GONI:Y         float64
GONI:PITCH     float64
GONI:YAW       float64
dtype: object

In [106]:
# convert RunNumber, DAQ:STATUS, CBREM:PLANE to int 
df_test['RunNumber'] = df_test['RunNumber'].astype(int)
df_test['DAQ:STATUS'] = df_test['DAQ:STATUS'].astype(int)
df_test['CBREM:PLANE'] = df_test['CBREM:PLANE'].astype(int)

In [107]:
# good runs comes from RCDB query @is_dirc_production and @status_approved
good_runs =[73266,73263,73262,73261,73260,73231,73230,73229,73228,73227,73226,73225,73224,73223,73222,73220,73219,73218,73217,73215,73214,73213,73212,73211,73210,73209,73208,73207,73191,73190,73188,73187,73186,73185,73184,73183,73182,73180,73179,73178,73177,73176,73175,73174,73173,73172,73171,73169,73168,73167,73166,73165,73164,73163,73162,73161,73160,73151,73150,73149,73148,73147,73146,73145,73144,73143,73132,73131,73130,73129,73127,73126,73125,73124,73123,73122,73121,73120,73119,73118,73117,73116,73115,73112,73108,73104,73102,73099,73096,73095,73094,73093,73092,73090,73089,73088,73087,73086,73085,73083,73082,73081,73079,73078,73077,73076,73075,73074,73073,73072,73071,73070,73069,73068,73067,73066,73065,73064,73063,73062,73061,73060,73059,73058,73057,73056,73055,73054,73053,73052,73051,73050,73049,73048,73047,73045,73044,73043,73042,73041,73040,73039,73038,72948,72947,72946,72944,72930,72929,72928,72927,72926,72925,72924,72923,72922,72921,72920,72919,72917,72916,72912,72911,72910,72909,72897,72890,72889,72888,72887,72883,72882,72881,72880,72879,72878,72872,72871,72870,72869,72868,72867,72864,72863,72862,72861,72860,72859,72858,72857,72855,72854,72852,72851,72850,72849,72848,72847,72846,72845,72844,72843,72842,72841,72840,72838,72837,72836,72835,72834,72833,72832,72831,72830,72829,72828,72827,72826,72825,72824,72823,72822,72816,72815,72814,72813,72812,72811,72801,72800,72799,72798,72797,72796,72795,72794,72793,72792,72791,72790,72789,72788,72786,72785,72783,72782,72781,72780,72779,72778,72776,72775,72774,72773,72772,72771,72770,72768,72763,72762,72761,72760,72759,72757,72756,72754,72753,72752,72751,72750,72749,72748,72747,72746,72745,72744,72743,72742,72741,72740,72739,72738,72737,72736,72734,72733,72732,72731,72730,72729,72728,72726,72725,72723,72722,72721,72720,72719,72717,72715,72714,72713,72710,72708,72707,72705,72704,72703,72702,72695,72690,72689,72688,72686,72684,72680,72679,72678,72677,72676,72673,72672,72670,72669,72668,72667,72666,72665,72663,72662,72661,72659,72658,72657,72656,72655,72654,72653,72651,72650,72649,72646,72645,72435,72434,72431,72428,72427,72426,72424,72423,72422,72420,72419,72418,72416,72414,72413,72412,72411,72410,72409,72408,72407,72406,72405,72404,72403,72401,72399,72398,72396,72395,72394,72393,72392,72391,72390,72389,72388,72387,72386,72385,72383,72381,72380,72379,72378,72377,72376,72375,72374,72372,72371,72370,72369,72368,72367,72366,72365,72364,72363,72362,72303,72302,72301,72299,72298,72297,72296,72295,72294,72293,72285,72284,72283,72282,72281,72280,72274,72273,72272,72267,72266,72265,72264,72263,72262,72261,72260,72259,72258,72257,72256,72251,72250,72249,72248,72247,72246,72245,72243,72241,72239,72238,72197,72196,72195,72194,72193,72191,72190,72189,72188,72186,72185,72184,72182,72181,72180,72179,72178,72177,72176,72175,72173,72172,72169,72168,72167,72166,72165,72164,72162,72160,72159,72158,72157,72156,72155,72154,72151,72149,72148,72147,72146,72145,72144,72142,72141,72138,72137,72136,72135,72134,72133,72132,72131,72130,72129,72128,72127,72126,72125,72124,72122,72121,72120,72119,72118,72117,72116,72114,72112,72111,72110,72109,72108,72107,72106,72105,72103,72102,72101,72100,72099,72095,72094,72092,72091,72090,72089,72087,72086,72085,72084,72083,72082,72081,72080,72079,72078,72077,72076,72075,72074,72073,72071,72070,72068,72067,72066,72065,72064,72063,72062,72061,72060,72059,72058,72057,72056,72055,72053,72052,72050,72048,72047,72046,72045,72043,72042,72041,72040,72039,72038,72037,72036,72034,72033,72031,72019,72018,72017,72016,72014,72013,72012,72011,72010,72009,72008,72007,72005,72004,72003,71999,71994,71993,71992,71991,71990,71982,71981,71980,71979,71978,71977,71976,71973,71972,71971,71970,71969,71968,71953,71952,71951,71950,71949,71947,71946,71945,71944,71943,71942,71940,71939,71938,71937,71936,71935,71934,71933,71927,71926,71925,71924,71923,71922,71921,71920,71919,71918,71916,71913,71912,71911,71910,71909,71908,71907,71906,71905,71904,71899,71898,71897,71896,71895,71894,71892,71891,71890,71889,71888,71887,71886,71885,71884,71883,71882,71881,71880,71879,71878,71877,71876,71875,71873,71872,71871,71870,71869,71868,71867,71865,71864,71863,71855,71854,71853,71852,71851,71850,71849,71848,71847,71846,71842,71841,71840,71839,71838,71837,71836,71835,71834,71833,71832,71831,71830,71827,71826,71825,71824,71823,71822,71821,71820,71819,71817,71815,71814,71813,71810,71809,71808,71807,71806,71800,71795,71794,71793,71792,71791,71790,71789,71786,71785,71784,71783,71782,71781,71780,71778,71777,71776,71775,71774,71773,71772,71770,71769,71768,71767,71766,71765,71763,71762,71761,71760,71759,71758,71757,71755,71754,71753,71752,71751,71748,71747,71743,71742,71740,71739,71737,71736,71735,71734,71729,71728,71727,71725,71724,71722,71721,71719,71718,71717,71716,71715,71714,71705,71704,71703,71701,71700,71699,71698,71697,71696,71695,71694,71693,71692,71691,71690,71689,71687,71686,71685,71684,71683,71681,71680,71679,71678,71677,71676,71675,71674,71672,71670,71669,71668,71667,71666,71663,71662,71661,71658,71656,71655,71654,71653,71652,71649,71648,71647,71646,71645,71644,71643,71641,71640,71638,71637,71635,71632,71631,71630,71629,71628,71627,71626,71625,71623,71619,71618,71617,71615,71614,71612,71603,71598,71597,71596,71594,71593,71592,71591,71590,71589,71588,71587,71586,71585,71563,71562,71561,71560,71559,71558,71557,71556,71554,71553,71551,71550,71545,71544,71543,71540,71539,71538,71537,71536,71535,71530,71529,71528,71527,71526,71525,71524,71522,71521,71520,71519,71518,71517,71516,71515,71514,71512,71511,71510,71502,71501,71500,71498,71497,71494,71493,71492,71491,71490,71487,71486,71485,71484,71483,71482,71478,71477,71475,71474,71473,71472,71470,71469,71468,71467,71466,71465,71464,71463,71462,71432,71431,71430,71429,71428,71406,71405,71404,71402,71401,71400,71396,71395,71382,71378,71377,71375,71373,71372,71371,71370,71369,71368,71365,71364,71363,71362,71358,71356,71355,71354,71353,71352,71351,71350]


In [108]:
# set a goodRun flag equal to whether or not RunNumber is in the set of good runs 
df_test['goodRun'] = df_test['RunNumber'].isin(good_runs)


In [109]:
print(df_test[['RunNumber','goodRun']])

       RunNumber  goodRun
0          71351     True
1          71351     True
2          71351     True
3          71351     True
4          71351     True
...          ...      ...
17337      73267    False
17338      73267    False
17339      73267    False
17340      73267    False
17341      73267    False

[17342 rows x 2 columns]


In [110]:
print(df_test.head())

             Date-Time  RunNumber  DAQ:STATUS  CBREM:PLANE  GONI:ROLL  GONI:X  \
0  2020-01-11T01:21:06      71351           0            0        0.0   164.5   
1  2020-01-11T01:21:55      71351           0            0        0.0   164.5   
2  2020-01-11T01:21:57      71351           0            0        0.0   164.5   
3  2020-01-11T01:22:23      71351           0            2        0.0   164.5   
4  2020-01-11T01:24:22      71351           0            2        0.0   164.5   

   GONI:Y  GONI:PITCH  GONI:YAW  goodRun  
0   13.19    0.354557  0.773983     True  
1   13.19    0.354557  0.773983     True  
2   13.19    0.354557  0.773983     True  
3   13.19    0.354557  0.773983     True  
4   13.19    1.830760  0.855300     True  


In [112]:
df_test.describe()

,RunNumber,DAQ:STATUS,CBREM:PLANE,GONI:ROLL,GONI:X,GONI:Y,GONI:PITCH,GONI:YAW
count,17342.000000,17342.000000,17342.000000,17342.000000,17342.000000,17342.000000,17342.000000,17342.000000
mean,72136.116769,0.615904,1.011879,6.878564,143.364508,58.392777,0.493379,1.522049
std,572.333301,0.774147,0.778271,20.297506,39.979366,23.817149,1.287622,1.352042
min,71351.000000,0.000000,0.000000,-21.500000,0.000000,0.000000,-5.000000,-5.000000
25%,71622.000000,0.000000,0.000000,-10.500000,142.500000,58.100000,-0.734710,0.773983
50%,71959.000000,0.000000,1.000000,0.000000,156.000000,72.000000,0.377610,1.000000
75%,72683.000000,1.000000,2.000000,34.500000,161.500000,73.500000,0.764000,2.441610
max,73267.000000,2.000000,2.000000,50.000000,165.000000,76.300000,9.000000,9.000000


In [111]:
df_run = df_test[df_test['RunNumber']==71351]

print(df_run.describe())
print(df_run.head(22))

       RunNumber  DAQ:STATUS  CBREM:PLANE  GONI:ROLL  GONI:X  GONI:Y  \
count       22.0   22.000000    22.000000       22.0    22.0   22.00   
mean     71351.0    0.500000     1.045455        0.0   164.5   13.19   
std          0.0    0.597614     0.785419        0.0     0.0    0.00   
min      71351.0    0.000000     0.000000        0.0   164.5   13.19   
25%      71351.0    0.000000     0.250000        0.0   164.5   13.19   
50%      71351.0    0.000000     1.000000        0.0   164.5   13.19   
75%      71351.0    1.000000     2.000000        0.0   164.5   13.19   
max      71351.0    2.000000     2.000000        0.0   164.5   13.19   

       GONI:PITCH   GONI:YAW  
count   22.000000  22.000000  
mean     1.226859   0.818338  
std      0.742879   0.041443  
min      0.354557   0.773983  
25%      0.354557   0.773983  
50%      1.830760   0.855300  
75%      1.830760   0.855300  
max      1.830760   0.855300  
              Date-Time  RunNumber  DAQ:STATUS  CBREM:PLANE  GONI:ROLL  

In [113]:
# need to change run number, so that run number for setup is same as for the actual data acquisition 
# currently, run number only changes when DAQ:STATUS is 1, which is after radiator setup. 

# need to loop through runs. For each run, should see the sequence 0 -> 1 -> 2 for DAQ:STATUS. Once it goes 2->0, next run should start 

df_out = pd.DataFrame()

run_list = df_test['RunNumber'].unique()
print(run_list)

for run_idx in range(len(run_list)):

    runVal = run_list[run_idx]

    if runVal == max(run_list):
        continue

    df_run = df_test[df_test['RunNumber']==runVal]

    daq_on = False
    for i, row in df_run.iterrows():
        #print(i)
        daq_status = row['DAQ:STATUS']
        #print("RunNumber:",row['RunNumber']," DAQ:STATUS:",daq_status)
        if daq_status == 2:
            #print("daq turned on")
            daq_on = True 
        
        if daq_on and daq_status != 2:
            #print("daq has been on, but not currently on")
            #print("currently assigned run number",runVal)
            #print("should reassign to run",run_list[run_idx+1])

            # need to find index for original data frame
            df_run.at[i,'RunNumber'] = run_list[run_idx+1]
    
    df_out = pd.concat([df_out,df_run])




[71351 71352 71353 ... 73265 73266 73267]


In [114]:
print(df_out.describe())

          RunNumber    DAQ:STATUS   CBREM:PLANE     GONI:ROLL        GONI:X  \
count  17334.000000  17334.000000  17334.000000  17334.000000  17334.000000   
mean   72136.162398      0.615553      1.012000      6.884767    143.402723   
std      571.838531      0.773935      0.778375     20.299841     39.913658   
min    71351.000000      0.000000      0.000000    -21.500000      0.000000   
25%    71623.000000      0.000000      0.000000    -10.500000    142.500000   
50%    71958.500000      0.000000      1.000000      0.000000    156.000000   
75%    72683.000000      1.000000      2.000000     34.500000    161.500000   
max    73267.000000      2.000000      2.000000     50.000000    165.000000   

             GONI:Y    GONI:PITCH      GONI:YAW  
count  17334.000000  17334.000000  17334.000000  
mean      58.402766      0.493946      1.521552  
std       23.805016      1.287649      1.352156  
min        0.000000     -5.000000     -5.000000  
25%       58.100000     -0.734710     

In [123]:
# now define the nudge sequences for each run 
run_list = df_out['RunNumber'].unique()

df_final_out = pd.DataFrame()

prevYaw = -1
prevPitch = -1
prevRoll = -1


df_out['NudgeDuringRun'] = False# only true if there was a nudge and DAQ is on (status 2)
df_out['NudgeNumber'] = 0
df_out['NudgePitchSize'] = 0.0
df_out['NudgeYawSize'] = 0.0

for run_idx in range(len(run_list)):
    runVal = run_list[run_idx]

    #if runVal > 71359:
    #    continue

    df_run = df_out[df_out['RunNumber']==runVal]

    num_nudges = 0

    print("RunNumber",runVal)


    for i, row in df_run.iterrows():


        if i==0 and run_idx==0:# could actually look into previous run even, would just need to check run_idx zero too
            prevYaw = row['GONI:YAW']
            prevPitch = row['GONI:PITCH']
            prevRoll = row['GONI:ROLL']

            continue

        currentYaw = row['GONI:YAW']
        currentPitch = row['GONI:PITCH']
        currentRoll = row['GONI:ROLL']

        yawDiff = abs(currentYaw - prevYaw)
        pitchDiff = abs(currentPitch - prevPitch)

        has_nudge = False # bool for if this row has a nudge event
        if yawDiff > 0.0001 or pitchDiff > 0.0001:
            num_nudges+=1
            #print("nudge number",num_nudges)
            has_nudge =True
            
            df_run.at[i,'NudgePitchSize'] = currentPitch - prevPitch
            df_run.at[i,'NudgeYawSize'] = currentYaw - prevYaw



        prevYaw = currentYaw 
        prevPitch = currentPitch
        prevRoll = currentRoll


        #print("daq_status",row['DAQ:STATUS'],"roll:",currentRoll,"pitch",currentPitch,"yaw",currentYaw)

        df_run.at[i,'NudgeNumber'] = num_nudges

        # check if daq on and nudge occurred
        if row['DAQ:STATUS']==2 and has_nudge:
            #print("Nudge occurred while the DAQ was on!!!")
            df_run.at[i,'NudgeDuringRun'] = True
        
    #print("total nudges for run is ",num_nudges)
    df_final_out = pd.concat([df_final_out,df_run])





RunNumber 71351
RunNumber 71352
RunNumber 71353
RunNumber 71354
RunNumber 71355
RunNumber 71356
RunNumber 71357
RunNumber 71358
RunNumber 71359
RunNumber 71360
RunNumber 71361
RunNumber 71362
RunNumber 71363
RunNumber 71364
RunNumber 71365
RunNumber 71366
RunNumber 71367
RunNumber 71368
RunNumber 71369
RunNumber 71370
RunNumber 71371
RunNumber 71372
RunNumber 71373
RunNumber 71374
RunNumber 71375
RunNumber 71376
RunNumber 71377
RunNumber 71378
RunNumber 71379
RunNumber 71380
RunNumber 71381
RunNumber 71382
RunNumber 71383
RunNumber 71384
RunNumber 71385
RunNumber 71386
RunNumber 71387
RunNumber 71388
RunNumber 71389
RunNumber 71390
RunNumber 71391
RunNumber 71392
RunNumber 71393
RunNumber 71394
RunNumber 71395
RunNumber 71396
RunNumber 71397
RunNumber 71398
RunNumber 71399
RunNumber 71400
RunNumber 71401
RunNumber 71402
RunNumber 71403
RunNumber 71404
RunNumber 71405
RunNumber 71406
RunNumber 71407
RunNumber 71408
RunNumber 71409
RunNumber 71410
RunNumber 71411
RunNumber 71412
RunNumbe

In [119]:
print(df_final_out.describe())

          RunNumber    DAQ:STATUS   CBREM:PLANE     GONI:ROLL        GONI:X  \
count  17334.000000  17334.000000  17334.000000  17334.000000  17334.000000   
mean   72136.162398      0.615553      1.012000      6.884767    143.402723   
std      571.838531      0.773935      0.778375     20.299841     39.913658   
min    71351.000000      0.000000      0.000000    -21.500000      0.000000   
25%    71623.000000      0.000000      0.000000    -10.500000    142.500000   
50%    71958.500000      0.000000      1.000000      0.000000    156.000000   
75%    72683.000000      1.000000      2.000000     34.500000    161.500000   
max    73267.000000      2.000000      2.000000     50.000000    165.000000   

             GONI:Y    GONI:PITCH      GONI:YAW   NudgeNumber  NudgeSize  \
count  17334.000000  17334.000000  17334.000000  17334.000000    17334.0   
mean      58.402766      0.493946      1.521552     48.121322        0.0   
std       23.805016      1.287649      1.352156    121.27702

In [120]:
df_final_out.to_csv("refined_data/runConfigInfo.csv")

In [121]:
import matplotlib.pyplot as plt 
import seaborn as sns

In [122]:
print(df_final_out['RunNumber'].max())

73267


In [35]:
df_run = df_final_out[df_final_out['RunNumber']==71351]

In [69]:
df_new = df_run[df_run['DAQ:STATUS']==2]

In [70]:
print(df_new['Date-Time'])

19    2020-01-11T01:33:45
Name: Date-Time, dtype: object


In [71]:
from datetime import datetime
on_time = datetime.fromisoformat(df_new['Date-Time'].item())

In [72]:
df_new = df_run[df_run['DAQ:STATUS']==1]
print(df_new.loc[11,'Date-Time'])
prestart_time = datetime.fromisoformat(df_new.loc[11,'Date-Time'])

2020-01-11T01:26:55


In [74]:
print(prestart_time - on_time)

-1 day, 23:53:10


In [90]:
df_new['GONI:ROLL'].unique()

array([0.])

In [84]:
time_diff = prestart_time-on_time 
print(time_diff.total_seconds())

-410.0


410.0
